r5py TTM code:

In [1]:
import osmnx as ox
import r5py
import os
from pathlib import Path
import geopandas as gpd
import pandas as pd
import datetime
from r5py import TravelTimeMatrixComputer

In [2]:
import sys
sys.argv.append(["--max-memory", "5G"])

In [3]:
# Set up the R5 network
osm_file = r'/mnt/d/Downloads/greater-london-latest.osm.pbf'
r5_network = r5py.TransportNetwork(osm_file)


In [6]:
msoa_london = pd.read_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa/processed/gld_msoa_centroids.csv', encoding='latin1')
# see if the column msoa21cd is unique:
msoa_london['msoa21cd'].nunique() == msoa_london.shape[0]

# follow this format: [{"lat": ..., "lng": ...}, ...]

# new dict origin: 1002 maoas
origin_coord_dict = {}
origin_coord = []

# new dict destination:
dest_coord = []
dest_coord_dict = {}

# for reference dict, read the msoa_london, use msoa21cd as key, lat and lon as value:
for i in range(msoa_london.shape[0]):
    msoaID = msoa_london.iloc[i, 3]
    lat = msoa_london.iloc[i, 8]
    lon = msoa_london.iloc[i, 9]
    #origin_coord.append({"lat": lat, "lng": lon})
    origin_coord_dict[msoaID] = {"lat": lat, "lng": lon}
    dest_coord_dict[msoaID] = {"lat": lat, "lng": lon}

In [12]:
# convert to dataframe, with msoa21cd as 'id':

origin_coord_df = pd.DataFrame(origin_coord_dict).T
origin_coord_df = origin_coord_df.reset_index()
origin_coord_df.columns = ['id', 'lat', 'lon']

In [13]:
origin_coord_df

,id,lat,lon
0,E02000318,51.488265,0.095618
1,E02000388,51.483157,-0.198445
2,E02000652,51.415041,-0.138046
3,E02000963,51.529079,-0.209371
4,E02000651,51.419153,-0.125942
...,...,...,...
997,E02000481,51.558124,0.293234
998,E02000885,51.512556,-0.053381
999,E02000674,51.441193,-0.001961
1000,E02000975,51.514949,-0.179854


In [15]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [16]:
origin_coord_df['geometry'] = origin_coord_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
origins_gdf = gpd.GeoDataFrame(origin_coord_df, geometry='geometry')
origins_gdf.set_crs(epsg=4326, inplace=True)  

,id,lat,lon,geometry
0,E02000318,51.488265,0.095618,POINT (0.09562 51.48827)
1,E02000388,51.483157,-0.198445,POINT (-0.19845 51.48316)
2,E02000652,51.415041,-0.138046,POINT (-0.13805 51.41504)
3,E02000963,51.529079,-0.209371,POINT (-0.20937 51.52908)
4,E02000651,51.419153,-0.125942,POINT (-0.12594 51.41915)
...,...,...,...,...
997,E02000481,51.558124,0.293234,POINT (0.29323 51.55812)
998,E02000885,51.512556,-0.053381,POINT (-0.05338 51.51256)
999,E02000674,51.441193,-0.001961,POINT (-0.00196 51.44119)
1000,E02000975,51.514949,-0.179854,POINT (-0.17985 51.51495)


In [18]:
# Initialize the travel time matrix computer
ttm_computer = TravelTimeMatrixComputer(
    r5_network,
    origins=origins_gdf,
    destinations=origins_gdf,
    departure=datetime.datetime(2024, 7, 23, 7, 30),
    transport_modes=["CAR"]
)

# Compute the travel time matrix
travel_time_matrix = ttm_computer.compute_travel_times()

# Convert the travel time matrix to a DataFrame
travel_time_df = pd.DataFrame(travel_time_matrix)

# Save results to a CSV file:
travel_time_df.to_csv("ttm_r5py.csv", index=False)


code dump:

In [ ]:
# Download OSM data for Greater London
place_name = "Greater London, England, United Kingdom"
graph = ox.graph_from_place(place_name, network_type='drive')

# Save the OSM data to a file
osm_file = Path("greater_london.osm.pbf")
ox.save_graphml(graph, filepath=osm_file)

# Set up the R5 network
r5_network = r5py.TransportNetwork(osm_file)